https://colab.research.google.com/drive/1hYF9rkWc-5Gz_QuTS-uK5lk6JVFM2TX2?usp=sharing

# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Pip install necessery library

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.0 MB/s eta 0:00:00


In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00


In [5]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ee68e0e64e5d3f6402adec80f4c7e16acce2f1663ba632797956fe9b465cec47
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [6]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Import necessery library

In [7]:
import torch
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import transformers
import wandb

import re
import nltk
from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

In [8]:
name = "test_run_best"

In [9]:
wandb.init(project="midterm_ds", name=name)

wandb: Currently logged in as: krprmf. Use `wandb login --relogin` to force relogin


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
data_train = pd.read_json('/content/drive/MyDrive/2110531-data-science-2023-01/train_for_student.json')
data_train = data_train.T
data_train.drop_duplicates(subset=['Title','Abstract'], inplace=True)
data_train.head()

,Title,Abstract,Classes
1,Activated carbon derived from bacterial cellul...,© 2019 Elsevier B.V.Activated carbon derived f...,"[CHE, MATENG]"
2,The algorithm of static hand gesture recogniti...,© Springer International Publishing AG 2018.Te...,[CPE]
3,Alternative Redundant Residue Number System Co...,© 2018 IEEE.Residue number system (RNS) is a n...,[EE]
4,Comparative study of wax inhibitor performance...,© Published under licence by IOP Publishing Lt...,"[PE, ME, CHE]"
5,Undrained lower bound solutions for end bearin...,"© 2019 John Wiley & Sons, Ltd.The undrained be...","[CE, MATSCI]"


In [12]:
data_train['Abstract'].iloc[0]

'© 2019 Elsevier B.V.Activated carbon derived from bacterial cellulose (BC-AC) was modified with various amounts of H3PO4(x wt% P/BC-AC) and used as a catalyst for the selective dehydration of ethanol to ethylene. The BC-AC obtained at a carbonization temperature of 500 °C had a mesoporous structure with surface area and total pore volume of ~1730 m2/g and 1.0 cm3/g, respectively. An increase in the H3PO4 loading from 5% to 40% increased the number of weak acid sites on the catalyst surface, which consequently enhanced ethanol conversion. At the reaction temperature of 400 °C, the modified BC-AC with 30-40 wt% H3PO4 loading (P/BC-AC) gave an ethanol conversion at 100% and an ethylene selectivity of 100%. A high selectivity for diethyl ether (DEE) at ~ 67% at ethanol conversion of ~ 50% was obtained at 200 °C. Stability tests with a time-on-stream of 12 h, at reaction temperatures of 200 and 400 °C, showed that the P/BC-AC catalyst had high thermal stability and stable catalytic activit

In [13]:
data_test = pd.read_json('/content/drive/MyDrive/2110531-data-science-2023-01/test_for_student.json')

data_test = data_test.T
data_test.head()

,Title,Abstract
001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...
002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...
003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...
004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...
005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...


In [14]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 454 entries, 1 to 454
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     454 non-null    object
 1   Abstract  454 non-null    object
 2   Classes   454 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


In [15]:
print(len(data_train),len(data_test))
print(len(data_train)/len(data_test),':',len(data_test)/len(data_test))


454 151
3.006622516556291 : 1.0


# Data preprocessing

In [16]:
df_test = data_test.copy()
cols = df_test.columns.tolist()
df_test['Text'] = df_test['Title']+". "+df_test['Abstract']
df_test = df_test.drop(columns=['Title', 'Abstract'])
# df.reset_index(drop=True, inplace=True)
df_test

,Text
001eval,Comparative Electrical Energy Yield Performanc...
002eval,Effects of graphene nanoplatelets on bio-based...
003eval,Anti-inflammatory action of two novel peptides...
004eval,Efficient all-and-one support vector machines ...
005eval,Driver identification using histogram and neur...
...,...
147eval,Utilization of Sewage Sludge from Beverage Ind...
148eval,Development of a Gateway for OpenADR-ECHONET L...
149eval,Effect of solution treatment and precipitation...
150eval,An effect-analysis method for species-dependen...


In [17]:
df = data_train.copy()
cols = df.columns.tolist()
df['Text'] = df['Title']+". "+df['Abstract']
df['labels'] = df['Classes']
df = df.drop(columns=['Title', 'Abstract','Classes'])
df.reset_index(drop=True, inplace=True)
df

,Text,labels
0,Activated carbon derived from bacterial cellul...,"[CHE, MATENG]"
1,The algorithm of static hand gesture recogniti...,[CPE]
2,Alternative Redundant Residue Number System Co...,[EE]
3,Comparative study of wax inhibitor performance...,"[PE, ME, CHE]"
4,Undrained lower bound solutions for end bearin...,"[CE, MATSCI]"
...,...,...
449,A portable USB-controlled potentiostat for pap...,"[CPE, CHE]"
450,Literature reviews on applying artificial inte...,"[CPE, EDU]"
451,A multi-parameterized water quality prediction...,"[ENV, EE, CHE]"
452,Semantic Segmentation on Medium-Resolution Sat...,"[EE, CPE, OPTIC, EDU]"


In [18]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
df['Text'] = df['Text'].apply(removeStopWords)
df_test['Text'] = df_test['Text'].apply(removeStopWords)

In [20]:
def text_preprocessing(text):

    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"-", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = ''.join(c for c in text if not c.isnumeric())
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [21]:
df['Text'] = df['Text'].apply(text_preprocessing)
df_test['Text'] = df_test['Text'].apply(text_preprocessing)

In [22]:
df

,Text,labels
0,activated carbon derived bacterial cellulose u...,"[CHE, MATENG]"
1,algorithm static hand gesture recognition usin...,[CPE]
2,alternative redundant residue number system co...,[EE]
3,comparative study wax inhibitor performance po...,"[PE, ME, CHE]"
4,undrained lower bound solutions end bearing ca...,"[CE, MATSCI]"
...,...,...
449,portable usb controlled potentiostat paper bas...,"[CPE, CHE]"
450,literature reviews applying artificial intelli...,"[CPE, EDU]"
451,multi parameterized water quality prediction m...,"[ENV, EE, CHE]"
452,semantic segmentation medium resolution satell...,"[EE, CPE, OPTIC, EDU]"


In [23]:
labels = df['labels']
mlb = MultiLabelBinarizer()
one_hot_labels = mlb.fit_transform(labels)

encoded_labels_df = pd.DataFrame(one_hot_labels, columns=mlb.classes_)
encoded_labels_df = encoded_labels_df.astype(int)
encoded_labels_df.head()

,AGRI,BME,CE,CHE,CPE,EDU,EE,ENV,IE,MATENG,MATH,MATSCI,ME,METAL,NANO,OPTIC,PE,SAFETY
0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [24]:
df = pd.concat([df['Text'], encoded_labels_df], axis=1)
df[list(mlb.classes_)] = df[list(mlb.classes_)].astype(int)
df.head()

,Text,AGRI,BME,CE,CHE,CPE,EDU,EE,ENV,IE,MATENG,MATH,MATSCI,ME,METAL,NANO,OPTIC,PE,SAFETY
0,activated carbon derived bacterial cellulose u...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,algorithm static hand gesture recognition usin...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,alternative redundant residue number system co...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,comparative study wax inhibitor performance po...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,undrained lower bound solutions end bearing ca...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [25]:
cat = list(mlb.classes_)

In [26]:
# Find rows where all values are 0
rows_all_zero = df.loc[(df.iloc[:, 1:] == 0).all(axis=1)]
print(f"Deleteable rows:{rows_all_zero.shape[0]}")

Deleteable rows:0


In [27]:
df_n = df.copy()

In [28]:
cols = df.columns.tolist()
labels = cols[1:]
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}
print(label2id)
print(id2label)

{'AGRI': 0, 'BME': 1, 'CE': 2, 'CHE': 3, 'CPE': 4, 'EDU': 5, 'EE': 6, 'ENV': 7, 'IE': 8, 'MATENG': 9, 'MATH': 10, 'MATSCI': 11, 'ME': 12, 'METAL': 13, 'NANO': 14, 'OPTIC': 15, 'PE': 16, 'SAFETY': 17}
{0: 'AGRI', 1: 'BME', 2: 'CE', 3: 'CHE', 4: 'CPE', 5: 'EDU', 6: 'EE', 7: 'ENV', 8: 'IE', 9: 'MATENG', 10: 'MATH', 11: 'MATSCI', 12: 'ME', 13: 'METAL', 14: 'NANO', 15: 'OPTIC', 16: 'PE', 17: 'SAFETY'}


In [29]:
df['one_hot_labels'] = df[labels].values.tolist()
df.head()

,Text,AGRI,BME,CE,CHE,CPE,EDU,EE,ENV,IE,MATENG,MATH,MATSCI,ME,METAL,NANO,OPTIC,PE,SAFETY,one_hot_labels
0,activated carbon derived bacterial cellulose u...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,algorithm static hand gesture recognition usin...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,alternative redundant residue number system co...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,comparative study wax inhibitor performance po...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,undrained lower bound solutions end bearing ca...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [30]:
df2 = df.copy()
df2.drop(inplace=True,columns=labels)
df2

,Text,one_hot_labels
0,activated carbon derived bacterial cellulose u...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,algorithm static hand gesture recognition usin...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,alternative redundant residue number system co...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,comparative study wax inhibitor performance po...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,undrained lower bound solutions end bearing ca...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
...,...,...
449,portable usb controlled potentiostat paper bas...,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
450,literature reviews applying artificial intelli...,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
451,multi parameterized water quality prediction m...,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
452,semantic segmentation medium resolution satell...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Model

## Split and Tokenize

In [31]:
df_train, df_val = train_test_split(df_n, random_state=32, test_size=0.2, shuffle=True)

train_dataset = Dataset.from_pandas(df_train,preserve_index=False)
val_dataset = Dataset.from_pandas(df_val,preserve_index=False)

dataset_dict = DatasetDict({"train": train_dataset, "val": val_dataset})
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Text', 'AGRI', 'BME', 'CE', 'CHE', 'CPE', 'EDU', 'EE', 'ENV', 'IE', 'MATENG', 'MATH', 'MATSCI', 'ME', 'METAL', 'NANO', 'OPTIC', 'PE', 'SAFETY'],
        num_rows: 363
    })
    val: Dataset({
        features: ['Text', 'AGRI', 'BME', 'CE', 'CHE', 'CPE', 'EDU', 'EE', 'ENV', 'IE', 'MATENG', 'MATH', 'MATSCI', 'ME', 'METAL', 'NANO', 'OPTIC', 'PE', 'SAFETY'],
        num_rows: 91
    })
})


In [32]:
example = dataset_dict['train'][0]
example

{'Text': 'identification zinc ion battery via equivalent circuit model. © ieee. notable zinc ion batteries (zibs) show coming trend replace li ion batteries (libs) future. intensive study provided measurable recognition trend. nevertheless, present work sets find prototype validate trend towards use zinc ion batteries. end, traditional equivalentcircuit model (ecm) combined recursive least squares (rls) method employed identify model. hence, model obtained via hybrid pulse power characterization test laboratory scale battery. results demonstrated excellent compatibility strategy dynamics zib. lead, therefore, efficient battery state prediction. strategy benefit battery management system (bms) development.',
 'AGRI': 0,
 'BME': 0,
 'CE': 0,
 'CHE': 0,
 'CPE': 1,
 'EDU': 0,
 'EE': 1,
 'ENV': 0,
 'IE': 0,
 'MATENG': 0,
 'MATH': 1,
 'MATSCI': 0,
 'ME': 0,
 'METAL': 1,
 'NANO': 0,
 'OPTIC': 0,
 'PE': 0,
 'SAFETY': 0}

In [33]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_length = 128

In [34]:
def tokenize_data(data):
  text = data["Text"]
  encode = tokenizer(text, padding="max_length", truncation=True, max_length=max_length)
  labels_batch = {k: data[k] for k in data.keys() if k in labels}
  labels_matrix = np.zeros((len(text), len(labels)))
  for id, label in enumerate(labels):
    labels_matrix[:, id] = labels_batch[label]
  encode["labels"] = labels_matrix.tolist()
  return encode

In [35]:
encoded_dataset = dataset_dict.map(tokenize_data, batched=True, remove_columns=dataset_dict['train'].column_names)

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [36]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [37]:
print(tokenizer.decode(example['input_ids']))
print(example['labels'])

[CLS] identification zinc ion battery via equivalent circuit model. © ieee. notable zinc ion batteries ( zibs ) show coming trend replace li ion batteries ( libs ) future. intensive study provided measurable recognition trend. nevertheless, present work sets find prototype validate trend towards use zinc ion batteries. end, traditional equivalentcircuit model ( ecm ) combined recursive least squares ( rls ) method employed identify model. hence, model obtained via hybrid pulse power characterization test laboratory scale battery. results demonstrated excellent compatibility strategy dynamics zib. lead, therefore, efficient battery state prediction. strategy benefit battery management system ( bms ) [SEP]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [38]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['CPE', 'EE', 'MATH', 'METAL']

In [39]:
encoded_dataset.set_format("torch")

## Set parameter of model

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=18,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# Best param
batch_size = 8
learning_rate = 1e-4
num_train_epochs = 35
weight_decay = 1e-4

In [43]:
training_args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="wandb"
)

## Define eval matrics
source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [44]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

def multi_label_metrics(predictions, labels):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= 0.5)] = 1
    y_true = labels
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

In [45]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [46]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  8720, 15813, 10163,  6046,  3081,  5662,  4984,  2944,  1012,
         1075, 15368,  1012,  3862, 15813, 10163, 10274,  1006,  1062, 12322,
         2015,  1007,  2265,  2746,  9874,  5672,  5622, 10163, 10274,  1006,
         5622,  5910,  1007,  2925,  1012, 11806,  2817,  3024,  2033, 28329,
         5038,  9874,  1012,  6600,  1010,  2556,  2147,  4520,  2424,  8773,
         9398,  3686,  9874,  2875,  2224, 15813, 10163, 10274,  1012,  2203,
         1010,  3151,  5662,  6895, 11890, 14663,  2944,  1006, 14925,  2213,
         1007,  4117, 28667,  9236,  3512,  2560, 14320,  1006,  1054,  4877,
         1007,  4118,  4846,  6709,  2944,  1012,  6516,  1010,  2944,  4663,
         3081,  8893,  8187,  2373, 23191,  3231,  5911,  4094,  6046,  1012,
         3463,  7645,  6581, 21778,  5656, 10949,  1062, 12322,  1012,  2599,
         1010,  3568,  1010,  8114,  6046,  2110, 17547,  1012,  5656,  5770,
         6046,  2968,  2291,  1006,  1038,  5244,  1007,   102])

In [47]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7284, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1950, -0.1695,  0.6317,  0.5563,  0.2733, -0.0532,  0.1044,  0.3165,
         -0.1370, -0.1583,  0.1360,  0.3820,  0.1209,  0.2387, -0.2273,  0.4234,
          0.0146, -0.2566]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [48]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train ans valtidate model

In [49]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.464356,0.032118,0.500000,0.000000
2,No log,0.458175,0.023946,0.500000,0.032967
3,No log,0.444279,0.023946,0.500000,0.032967
4,No log,0.444194,0.032118,0.500000,0.000000
5,No log,0.436547,0.000000,0.500000,0.000000
6,No log,0.451038,0.000000,0.500000,0.000000
7,No log,0.447088,0.000000,0.500000,0.000000
8,No log,0.431987,0.000000,0.500000,0.000000
9,No log,0.449051,0.000000,0.500000,0.000000
10,No log,0.440739,0.032118,0.500000,0.000000


TrainOutput(global_step=1610, training_loss=0.4246036505847244, metrics={'train_runtime': 547.4615, 'train_samples_per_second': 23.207, 'train_steps_per_second': 2.941, 'total_flos': 835826545128960.0, 'train_loss': 0.4246036505847244, 'epoch': 35.0})

In [50]:
trainer.evaluate()

{'eval_loss': 0.4643557071685791,
 'eval_f1': 0.03211805555555555,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.7498,
 'eval_samples_per_second': 121.358,
 'eval_steps_per_second': 16.003,
 'epoch': 35.0}

#Test and save

In [51]:
trainer.save_model("/content/drive/MyDrive/2110531-data-science-2023-01/best1")

In [52]:
checkpoint = "/content/drive/MyDrive/2110531-data-science-2023-01/best1"

In [53]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [54]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [55]:
predictions = []
for i in df_test['Text']:
  encoding = tokenizer(i, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logits.squeeze().cpu())
  prediction = np.zeros(probs.shape)
  prediction[np.where(probs >= 0.5)] = 1
  predictions.append(np.array(prediction))

In [56]:
len(predictions)

151

In [57]:
outputs = np.array(predictions)

In [58]:
outputs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
outputs[0,:]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [60]:
pred = {}
for i in range(len(outputs)):
  for j in range(len(labels)):
    if labels[j] not in pred:
      pred[labels[j]] = []
      pred[labels[j]].append(outputs[i,j])
      print(pred)
    else:
      pred[labels[j]].append(outputs[i,j])


{'AGRI': [0.0]}
{'AGRI': [0.0], 'BME': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0], 'EE': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0], 'EE': [0.0], 'ENV': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0], 'EE': [0.0], 'ENV': [0.0], 'IE': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0], 'EE': [0.0], 'ENV': [0.0], 'IE': [0.0], 'MATENG': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': [0.0], 'EDU': [0.0], 'EE': [0.0], 'ENV': [0.0], 'IE': [0.0], 'MATENG': [0.0], 'MATH': [0.0]}
{'AGRI': [0.0], 'BME': [0.0], 'CE': [0.0], 'CHE': [1.0], 'CPE': 

In [61]:
new_labels = ['CE','ENV','BME','PE','METAL','ME','EE','CPE','OPTIC','NANO','CHE','MATENG','AGRI','EDU','IE','SAFETY','MATH','MATSCI']

In [62]:
pred = {}
for i in range(len(outputs)):
  for j in range(len(labels)):
    if labels[j] not in pred:
      pred[labels[j]] = []
      pred[labels[j]].append(outputs[i,j])
    else:
      pred[labels[j]].append(outputs[i,j])


In [63]:
new_preds = {key: pred[key] for key in new_labels}
print(new_preds)

{'CE': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'ENV': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [64]:
data_test

,Title,Abstract
001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...
002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...
003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...
004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...
005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...
...,...,...
147eval,Utilization of Sewage Sludge from Beverage Ind...,© Published under licence by IOP Publishing Lt...
148eval,Development of a Gateway for OpenADR-ECHONET L...,"© 2018 IEEE.In this paper, we develop an ECHON..."
149eval,Effect of solution treatment and precipitation...,© 2017 Elsevier Ltd. All rights reserved.The a...
150eval,An effect-analysis method for species-dependen...,"© The Authors, published by EDP Sciences, 2019..."


In [65]:
data_test.index

Index(['001eval', '002eval', '003eval', '004eval', '005eval', '006eval',
       '007eval', '008eval', '009eval', '010eval',
       ...
       '142eval', '143eval', '144eval', '145eval', '146eval', '147eval',
       '148eval', '149eval', '150eval', '151eval'],
      dtype='object', length=151)

In [66]:
bf_df = data_test.copy()
df_pred= pd.DataFrame.from_dict(new_preds)
df_pred.set_index(data_test.index, inplace=True)
df_pred.reset_index(inplace = True)
df_pred.rename(columns = {'index':'id'}, inplace = True)
# df_pred['id'] = df_pred['index']

In [67]:
df_pred[labels] = df_pred[labels].astype(int)
df_pred

,id,CE,ENV,BME,PE,METAL,ME,EE,CPE,OPTIC,NANO,CHE,MATENG,AGRI,EDU,IE,SAFETY,MATH,MATSCI
0,001eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,002eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,003eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,004eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,005eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
147,148eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
148,149eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
149,150eval,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [68]:
df_pred.to_csv('/content/drive/MyDrive/2110531-data-science-2023-01/predict.csv', index=False)

In [69]:
df_verify = pd.read_csv('/content/drive/MyDrive/2110531-data-science-2023-01/predict.csv')
print(df_verify)

          id  CE  ENV  BME  PE  METAL  ME  EE  CPE  OPTIC  NANO  CHE  MATENG  \
0    001eval   0    0    0   0      0   0   0    0      0     0    1       0   
1    002eval   0    0    0   0      0   0   0    0      0     0    1       0   
2    003eval   0    0    0   0      0   0   0    0      0     0    1       0   
3    004eval   0    0    0   0      0   0   0    0      0     0    1       0   
4    005eval   0    0    0   0      0   0   0    0      0     0    1       0   
..       ...  ..  ...  ...  ..    ...  ..  ..  ...    ...   ...  ...     ...   
146  147eval   0    0    0   0      0   0   0    0      0     0    1       0   
147  148eval   0    0    0   0      0   0   0    0      0     0    1       0   
148  149eval   0    0    0   0      0   0   0    0      0     0    1       0   
149  150eval   0    0    0   0      0   0   0    0      0     0    1       0   
150  151eval   0    0    0   0      0   0   0    0      0     0    1       0   

     AGRI  EDU  IE  SAFETY  MATH  MATSC